In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
import time
import random
import os
import shutil
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver import Proxy
from selenium.webdriver.chrome.webdriver import WebDriver as ChromeDriver
from fake_useragent import UserAgent
from selenium.webdriver.support.wait import WebDriverWait

In [ ]:
df=pd.read_excel('./reference/url_machine_6.xlsx',header=None)

In [ ]:
links=df[0].tolist()

In [ ]:
def newChromeBrowserDefault(headless=True, downloadPath=None):
    """ Helper function that creates a new Selenium browser """
    options = webdriver.ChromeOptions()
    options.add_argument("user-agent="+UserAgent().random)
    if headless:
        options.add_argument('headless')
    if downloadPath is not None:
        prefs = {}
        if not os.path.exists(downloadPath):
            os.makedirs(downloadPath)
        prefs["profile.default_content_settings.popups"]=0
        prefs["download.default_directory"]=downloadPath
        options.add_experimental_option("prefs", prefs)

    browser = webdriver.Chrome(chrome_options=options, executable_path='C:\driver\chromedriver.exe')
    return browser

In [ ]:
def newChromeBrowserProxy(headless=True, downloadPath=None,proxy='186.216.81.21:31773'):
    """ Helper function that creates a new Selenium browser """
    options = webdriver.ChromeOptions()
    options.add_argument("user-agent="+UserAgent().random)
    if headless:
        options.add_argument('headless')
    if downloadPath is not None:
        prefs = {}
        if not os.path.isdir(downloadPath):
            os.makedirs(downloadPath)
        prefs["profile.default_content_settings.popups"]=0
        prefs["download.default_directory"]=downloadPath
        options.add_experimental_option("prefs", prefs)

    settings = {
    "httpProxy": proxy,
    "sslProxy": proxy
    }
    proxy = Proxy(settings)
    from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
    cap = DesiredCapabilities.CHROME.copy()
    cap['platform'] = "WINDOWS"
    cap['version'] = "7"
    proxy.add_to_capabilities(cap)

    browser = ChromeDriver("C:\driver\chromedriver.exe",desired_capabilities=cap,chrome_options=options)
    return browser

In [ ]:
len(links)==100000

In [ ]:
downloadPath="enter_download_path"

In [ ]:
driver = newChromeBrowserDefault(headless=False,downloadPath=downloadPath)
# Code 2
# driver = newChromeBrowserProxy(headless=False,downloadPath=downloadPath)

In [ ]:
def get_time(downloadPath):
    return max([os.path.getmtime(os.path.join(downloadPath, f)) for f in os.listdir(downloadPath) if os.path.isfile(os.path.join(downloadPath, f))], default=0)

def download_done_chk1(downloadPath):
    for f in os.listdir(downloadPath):
        if (".tmp" in f) or (".crdownload" in f) or (".part" in f):
            time.sleep(0.5)
            print("in recursion 1")
            download_done_chk1(downloadPath)
    
def download_done_chk2(previous_mtime):
    new_mtime=get_time(downloadPath)
    if new_mtime==previous_mtime:
        time.sleep(0.5)
        print("in recursion 2")
        download_done_chk2(previous_mtime)
    elif new_mtime<previous_mtime:
        print("unexpected error")

In [ ]:
offset=400000
previous_mtime=get_time(downloadPath)

for idx,link in enumerate(links[offset-400000:100]):
    driver.get(link.replace('.nl','.com'))
    try:
        element = WebDriverWait(driver, 10).until(lambda x: x.find_element_by_xpath('/html/body/div[2]/div[2]/div/md-content/div/div/div[1]/trends-widget/ng-include/widget/div/div/div/widget-actions/div/button[1]'))
    except:
        driver.refresh()
        try:
            element = WebDriverWait(driver, 10).until(lambda x: x.find_element_by_xpath('/html/body/div[2]/div[2]/div/md-content/div/div/div[1]/trends-widget/ng-include/widget/div/div/div/widget-actions/div/button[1]'))
        except:
            print("Exception occured: Verify if the daily limit is reached on current network.")
            
    print("Ready to download")
    element.click()
    download_done_chk1(downloadPath)
    download_done_chk2(previous_mtime)
    filename = max([os.path.join(downloadPath, f) for f in os.listdir(downloadPath) if os.path.isfile(os.path.join(downloadPath, f))], key=os.path.getmtime)
    shutil.move(filename,downloadPath+'\\'+"data"+"_"+str((offset+idx))+'.csv')
    previous_mtime=get_time(downloadPath)
    print(idx)
    print(link)
    print("Success")